In [1]:
import os
import sys
import logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # FATAL
logging.getLogger('tensorflow').setLevel(logging.DEBUG)

try:
    from google.colab import drive
    drive.mount('/content/drive')
    !pip install -q ruamel.yaml
    !pip install -q tensorboard-plugin-profile
    project_path = '/content/drive/MyDrive/Colab Projects/quantumflow'
except:
    project_path = os.path.expanduser('~/quantumflow')

In [2]:
os.chdir(project_path)
sys.path.append(project_path)

import tensorflow as tf
%load_ext tensorboard

import quantumflow

experiment = 'resnets'
run_name = 'resnet_100'

base_dir = os.path.join(project_path, "experiments", experiment)
params = quantumflow.utils.load_yaml(os.path.join(base_dir, f'{experiment}.yaml'))[run_name]
run_dir = os.path.join(base_dir, run_name)

In [3]:
## %tensorboard --logdir="$base_dir" --load_fast=false

In [4]:
dataset_train = quantumflow.instantiate(params['dataset_train'], run_dir=run_dir)
dataset_train.build()

dataset_validate = quantumflow.instantiate(params['dataset_validate'], run_dir=run_dir)
dataset_validate.build()

dataset dataset.hdf5 saved to /home/manuel/quantumflow/experiments/snyder_2012/recreate_dataset
dataset dataset.hdf5 saved to /home/manuel/quantumflow/experiments/snyder_2012/validate_dataset


In [5]:
tf.profiler.experimental.server.start(6009)
tf.keras.backend.clear_session()
tf.random.set_seed(params['seed'])

model = quantumflow.instantiate(params['model'], run_dir=run_dir, dataset=dataset_train)
display(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
density (InputLayer)            [(None, 500)]        0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 500, 1)       0           density[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 500, 32)      3232        lambda[0][0]                     
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 500, 32)      102432      conv1d[0][0]                     
______________________________________________________________________________________________

None

# Visualization

In [6]:
from quantumflow.utils import anim_plot
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import tree
%matplotlib inline

ModuleNotFoundError: No module named 'tree'

In [ ]:
sample_features = tree.map_structure(lambda feature: feature[10:11], dataset_validate.features)
sample_targets = tree.map_structure(lambda target: target[10:11], dataset_validate.targets)
sample_targets_pred = model(sample_features)
sample = tree.map_structure(lambda target, target_pred: (target[0], target_pred.numpy()[0]), sample_targets, sample_targets_pred)

In [ ]:
for target_path, (target, target_pred) in tree.flatten_with_path_up_to(sample_targets, sample):
    target_name = '/'.join(target_path)
    
    if np.squeeze(target).shape == dataset_validate.x.shape:

        plt.figure(figsize=(20, 3))
        plt.plot(dataset_validate.x, target, 'k:')
        plt.plot(dataset_validate.x, np.squeeze(target_pred))
        plt.title(target_name)
        plt.show()
    else:
        print(f"{target_name}: {target_pred} ({target})")

In [ ]:
value = None
tensors = {}

def plot_layer(layer):
    global value, tensors
    
    if isinstance(layer, tf.keras.Model):
        tree.traverse(plot_layer, layer.layers)

    elif isinstance(layer, tf.keras.layers.InputLayer):
        value = sample_features[layer.name]
        tensors[layer.output.name] = value
        
    elif isinstance(layer, tf.keras.layers.Layer):
        if isinstance(layer.input, list):
            value = layer([tensors[inp.name] for inp in layer.input])
        else:
            value = layer(tensors[layer.input.name])
        tensors[layer.output.name] = value

_ = tree.traverse(plot_layer, model.layers)

In [ ]:
for layer_name, tensor in tensors.items():
    if len(tensor.shape) == 3:
        plt.figure(figsize=(20, 3))
        plt.plot(tensor[0])
        plt.title(layer_name)
        plt.show()
    elif np.prod(tensor.shape) < 100:
        print(layer_name, tensor)
    else:
        print(layer_name, tensor.shape)

# Train

In [ ]:
raise YouShallNotPass

In [7]:
optimizer = quantumflow.instantiate(params['optimizer'])

model.compile(
    optimizer,
    loss=params['loss'], 
    loss_weights=params.get('loss_weights', None), 
    metrics=params.get('metrics', None)
)


if params.get('load_checkpoint', None) is not None:
    model.load_weights(os.path.join(data_dir, params['load_checkpoint']))
    if params['fit'].get('verbose', 0) > 0:
        print("loading weights from ", os.path.join(data_dir, params['load_checkpoint']))

callbacks = []


if run_dir is not None and params.get('checkpoint', False):
    checkpoint_params = params['checkpoint'].copy()
    checkpoint_params['filepath'] = os.path.join(run_dir, checkpoint_params.pop('filename', 'weights.{epoch:05d}.hdf5'))
    checkpoint_params['verbose'] = checkpoint_params.get('verbose', min(1, params['fit'].get('verbose', 1)))
    callbacks.append(tf.keras.callbacks.ModelCheckpoint(**checkpoint_params))


if 'tensorboard' in params:
    callbacks.append(
        quantumflow.instantiate(params['tensorboard'], log_dir=run_dir, learning_rate=optimizer.learning_rate))


model.fit(x=dataset_train.features, 
          y=dataset_train.targets, 
          callbacks=callbacks,
          validation_data=(dataset_validate.features, dataset_validate.targets) if dataset_validate is not None else None,
          **params['fit'])

if params['save'] is True:
    save_model = getattr(model, params['save_model']) if not params.get('save_model', 'self') == 'self' else model
    save_model.save(os.path.join(run_dir, 'saved_model'), include_optimizer=False)

INFO:tensorflow:Assets written to: /home/manuel/quantumflow/experiments/resnets/resnet_100/saved_model/assets


/home/manuel/.local/lib/python3.8/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
